# 认证
- 认证和权限的相关知识：http://www.hyhblog.cn/2018/04/25/user_login_auth_terms/
- 认证可以简单的理解为登录和访问需要登录的接口的认证，只要认证通过了，那么在request对象（是drf的对象上便有两个属性），一个是request.user，一个是request.auth，前者是django中的User对象，后者根据不同的认证机制有不同的对象。DRF内置了几个认证的模块，以下：

## rest_framework.authentication.BasicAuthenitcation
- 基本的授权，每次都要在header中把用户名和密码传给服务器，因此不是很全全，不能在生产环境中使用g

## rest_framework.authtication.SessionAuthentication：
- 基于django的session机制实现的，如果前端部分是网页，那么用它是可以的，如果前端是iOS或者Android的app，那它用起来就不太方便

## rest_framework.authentication.TokenAuthentication
- 基于token的认证机制，只要登录完成后便会返回一个token，以后请求一些需要登录的api，就通过传递这个token就可以了，并且这个token是存储在服务器的数据库中的
- 但是这种token的方式有一个缺点，它没有自动过期机制，一旦登录完成后，这个token是永久有效的，这个是不安全的


# JSON Web Token认证机制
- JSON Web Token简称JWT，在前后端分离的项目中，或者是APP的项目中，推荐使用JWT，JWT是在登录成功后，把用户的相关信息以及过期时间进行加密，然后生成一个token返回给客户端，客户端拿到后可以存储起来，以后每次请求的时候都携带这个token，服务器在接收到需要登录的api请求后，对这个token进行解密，然后获取过期时间和用户信息（比如用户id），如果过期了或者用户信息不对，那么都是认证失败，JWT的相关代码如下：
        import jwt
        from django.conf import settings
        from rest_framework.authentication import BaseAuthentication,get_authorization_header
        from rest_framework import exceptions
        from django.contrib.auth import get_user_model
        from jwt.exceptions import ExpiredSignatureError
        MTUser = get_user_model()
        import time

        def generate_jwt(user):
            expire_time = int(time.time() + 60*60*24*7)
            return jwt.encode({"userid":user.pk,"exp":expire_time},key=settings.SECRET_KEY).decode('utf-8')


        class JWTAuthentication(BaseAuthentication):
            keyword = 'JWT'
            def authenticate(self, request):
                auth = get_authorization_header(request).split()

                if not auth or auth[0].lower() != self.keyword.lower().encode():
                    return None

                if len(auth) == 1:
                    msg = "不可用的JWT请求头！"
                    raise exceptions.AuthenticationFailed(msg)
                elif len(auth) > 2:
                    msg = '不可用的JWT请求头！JWT Token中间不应该有空格！'
                    raise exceptions.AuthenticationFailed(msg)

                try:
                    jwt_token = auth[1]
                    jwt_info = jwt.decode(jwt_token,settings.SECRET_KEY)
                    userid = jwt_info.get('userid')
                    try:
                        # 绑定当前user到request对象上
                        user = MTUser.objects.get(pk=userid)
                        return user, jwt_token
                    except:
                        msg = '用户不存在！'
                    raise exceptions.AuthenticationFailed(msg)
                except ExpiredSignatureError:
                    msg = "JWT Token已过期！"
                    raise exceptions.AuthenticationFailed(msg)

# 配置认证
- 配置认证方式有两种，一种是全局的，在settings.REST_FRAMEWORK.DEFAULT_AUTHENTICATION_CLASSES中配置，第二种就是在需要认证的视图中，通过authentication_class进行配置，如下：
        REST_FRAMEWORK = {
            'DEFAULT_AUTHENTICATION_CLASSES': ['apps.mtauth.authentications.JWTAuthentication'],
            'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
            'PAGE_SIZE': 12,
            'PAGE_QUERY_PARAM': "page"
        }
        

# 权限
- 不同的api拥有不同的访问权限，比如普通用户有读文章的权限，但是没有删除文章的权限，因此需要用到权限来进行API的管理，以下是DRF自带的权限

## permissions.AllowAny
- 允许所有人访问

## permissions.IsAuthenticated
- 是登录的用户即可访问（判断条件是request.user and request.user.is_authenticated）

## permissions.IsAdminUser
- 是管理员（判断条件是request.user and user.is_staff）

## permissions.IsAuthenticatedOrReadOnly
- 是登录的用户，并且这个API是只能读的（也就是GET, OPTIONS. HEAD）


# 自定义权限
- 有时候drf自带的权限无法满足要求，那么我们可以自定义权限，自定义权限要遵循两个条件
    - 1.继承自permissions.BasePermission
    - 2.实现has_permisssion(self,request，view)或者是has_object_permission(self,request,view,obj)方法，第一个方法是管理整个视图的访问权限，第二个访问时用来管理某个对象的访问权限（比如只能修改自己的用户信息）
    
            from rest_framework import permissions
        class IsOwnerOrReadOnly(permissions.BasePermission):
            def has_object_permission(self, request, view, obj):
                if request.method in permissions.SAFE_METHODS:
                    return True
                return obj.owner == request.user
                
## 权限的使用
- 权限的使用也就是两种方式，第一种是在settingsREST_FRAMEWORK.DEFAULT_PERMISSION_CLASSES设置，第二种是在具体的视图函数中通过permission_classes来设置
        # settings.py
        REST_FRAMEWORK = {
            'DEFAULT_PERMISSION_CLASSES': [
                'rest_framework.permissions.IsAuthenticated',
            ]
        }

        # views.py
        class ExampleView(APIView):
            permission_classes = [IsAuthenticated]
            def get(self, request, format=None):
                content = {
                    'status': 'request was permitted'
                }
                return Response(content)